<a href="https://colab.research.google.com/github/eljimenezj/CM0868--Estadistica-Multivariada-Avanzada/blob/master/02_Clasificaci%C3%B3n_de_imagenes_DeepLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Nota: ESTE LIBRO AUN SE ENCUENTRA EN PROCESO DE CONSTRUCCIÓN** - Marzo 23 de 2020

# Estadistica Multivariada - Clasificación de imagenes con modelos de Deep  Learning 

Este libro contiene un estudio de de clasificación de imagenes utilizando modelos de redes neuronales profundas. Esto libro hace parte del proyecto final del curso de Estadistica Multivariada Avanzanda de la Maestría de Ciencia de Datos.

Los integrantes de este trabajo:

* Jorge Luis Renteria Roa
* Edgar Leandro Jimenez Jaimes





Vamos a instalar las librerias necesarias para la ejecución de este libro

In [1]:
from PIL import Image
import pandas as pd
import glob
import re
import numpy as np
import imutils
import time
from google.colab import drive
from google.colab import files
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from keras.preprocessing import image
import cv2 as cv
from pathlib import Path
from sklearn.model_selection import GridSearchCV, train_test_split
from skimage.io import imread
import warnings
warnings.filterwarnings('ignore')


import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator


pd.set_option('display.max_columns', 30*30 + 1)

Using TensorFlow backend.


In [0]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## Lectura y estructuración de pipeline

In [0]:
dir_train = '/content/drive/My Drive/Estadistica Multivariada/Proyecto Final/Dataset/datalake_rente/Train/'
datagen_train = ImageDataGenerator(rescale=1./255)

dir_val = '/content/drive/My Drive/Estadistica Multivariada/Proyecto Final/Dataset/datalake_rente/Validation/'
datagen_val = ImageDataGenerator(rescale=1./255)

gen_train = datagen_train.flow_from_directory(
    dir_train, target_size=(256, 256), class_mode='sparse'
)
gen_val = datagen_val.flow_from_directory(
    dir_val, target_size=(256,256), class_mode='sparse' # Usar el mismo numerode pixeles que en el modelo clasico
)

Found 30 images belonging to 2 classes.
Found 8 images belonging to 2 classes.


In [0]:
# Leyendo un modelo h5 '/content/drive/
 
 # Recreate the exact same model, including its weights and the optimizer
#new_model = tf.keras.models.load_model('modelo1.h5')

# Show the model architecture
#new_model.summary()

# Clasificación de imagenes utilizando utilizando redes neuronales profundas

Para esto utilizaremos los 3 modelos:

1. Red neuronal Fully Conected
2. Red Neuronal Convolucional con arquitectura propuesta
3. Red Neuronal Convolucional con arquitectura predefinida (Resnet de Keras)

## Modelo 1 - Red neuronal Fully Conected

In [0]:
modelo1 = tf.keras.models.Sequential([
    # El tamaño esperado es 150x150 con 3 bytes de color
    tf.keras.layers.Flatten(input_shape=(400, 400, 3)),       # Usar el mismo numerode pixeles que en el modelo clasico
    # 512 neuronas en la capa interna
    tf.keras.layers.Dense(512, activation='relu'),            # Primera capa (Potencias del dos)
    tf.keras.layers.Dropout(0.5),                             # Regularizacion capa 1
    tf.keras.layers.Dense(256, activation='relu'),            # Segunda capa
    tf.keras.layers.Dropout(0.5),                             # Regularizacion capa 2
    tf.keras.layers.Dense(128, activation='relu'),            # Tercera capa
    tf.keras.layers.Dropout(0.5),                             # Regularizacion capa 3
    # Probabilidad de pertenencia a cada una de las 3 clases
    tf.keras.layers.Dense(2, activation='softmax')
])

modelo1.summary() # Mirar arquitectura del modelo

modelo1.compile(loss='sparse_categorical_crossentropy',      # Funcion de perdida
                optimizer='adam',                            # Optimizar los pesos de la red
                metrics=['accuracy'])                        # Ojo con las clases desbalanceadas

In [0]:
# Entrenamiento
historia1 = modelo1.fit_generator(gen_train,                 # Data de entrenamiento 
                                  epochs=5,                  # Cuantas veces va a pasar por el dataset (100 podría ser un numero)
                                  validation_data=gen_val,   # Data de validacion
                                  steps_per_epoch=10,        # Particiones para entrenar
                                  verbose=1)                 # Mostar los logs de entrenamiento
# Guardo el modelo
modelo1.save("modelo1.h5")

In [0]:
## Mostarmos rendimientos en train y test de la red por cada epoca.

import matplotlib.pyplot as plt

def plot_history(history):
  acc = history.history['acc']
  val_acc = history.history['val_acc']
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  epocas = range(len(acc))

  plt.plot(epocas, acc, 'r', label='Entrenamiento')
  plt.plot(epocas, val_acc, 'b', label='Validación')
  plt.title('Precisión en cada época')
  plt.legend(loc=0)
  
plot_history(historia1)

## Modelo 2 - Red Neuronal Convolucional con arquitectura propuesta

Revisar
https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html

## Modelo 3 -  Red Neuronal Convolucional con arquitectura predefinida (Resnet de Keras)

In [0]:
# Instacia del modelo y configuración
base_model = tf.keras.applications.resnet50.ResNet50(weights= 'imagenet', 
                                            include_top=False, 
                                            input_shape= (256,256,3))

x = base_model.output                                       # salida de resnet
x = tf.keras.layers.GlobalAveragePooling2D()(x)             # operacion de pooling
x = tf.keras.layers.Dropout(0.7)(x)                         # regularizar

predictions = tf.keras.layers.Dense(2,                      # numero de clases
                    activation= 'softmax')(x)

model = tf.keras.models.Model(inputs = base_model.input, outputs = predictions)

#

model.summary() # Mirar arquitectura del modelo resnet50

model.compile(loss='sparse_categorical_crossentropy',        # Funcion de perdida
                optimizer='adam',                            # Optimizar los pesos de la red
                metrics=['accuracy'])                        # Ojo con las clases desbalanceadas

In [0]:
# Entrenamiento
historia2 = model.fit_generator(gen_train,                  # Data de entrenamiento 
                                  epochs=5,                 # Cuantas veces va a pasar por el dataset (100 podría ser un numero)
                                  validation_data=gen_val,  # Data de validacion
                                  steps_per_epoch=50,       # Particiones para entrenar
                                  #batch_size=16,           #
                                  verbose=1)                # Mostar los logs de entrenamiento
# Guardo el modelo
model.save("modelo2.h5")

**TIPS**

* Aplicar las mismas transformaciones tanto para los modelos clasicos como los modelos de DL.

* Se espera que de menor el transfer learning sea mejor.

* Para el modelo de cero replicar el link

* Para el fully conected (no va a dar tan bien). Bajar capas.

* Tamaño: Intentar con 300, entre mas grande mas informacion tiene



In [0]:
# Repo de Imagenes
path= '/content/drive/My Drive/Estadistica Multivariada/Proyecto Final/Dataset/DataAUG'